# Running history animation

This notebook loads in a history of GPS data over a particular geographic area and creates an animation of it.
Enjoy!

In [ ]:
#Importing and function definitions

import gpxpy
#import os
#os.chdir('/home/evans/runningmaps/')
import glob
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import contextily as ctx
from pyproj import Transformer
from matplotlib.lines import Line2D
import datetime
import matplotlib.patheffects as PathEffects

#Extract the timestamp, latitude, and longitude of each run
def extract_gpx_data(gpx_path):
    with open(gpx_path, 'r') as f:
        gpx = gpxpy.parse(f)
    
    gps = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                timestamp = point.time.replace(tzinfo=timezone.utc)
                gps.append((timestamp, point.latitude, point.longitude))
    return gps

#Align the runs so that each run starts at the same time.
#Calculate the instantaneous speed at each point in the run
def time_aligned_track(track):
    
    start_time = track[0][0]

    times = [(t - start_time).total_seconds() for t, _, _ in track]
    geometries = [Point(lon, lat) for _, lat, lon in track]

    gdf = gpd.GeoDataFrame(
        {'time': times, 'timestamp': [t for t, _, _ in track]},
        geometry=geometries,
        crs='EPSG:4326'
    ).to_crs(epsg=3857)

    return gdf

# Convert the axis labels from meters to degrees
def meters_to_lonlat(ax):
    
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", \
                                        always_xy=True)

    def format_x(x, _):
        lon, _ = transformer.transform(x, ax.get_ylim()[0])
        
        return f"{lon:.2f}°"

    def format_y(y, _):
        _, lat = transformer.transform(ax.get_xlim()[0], y)
        
        return f"{lat:.2f}°"

    ax.xaxis.set_major_formatter(plt.FuncFormatter(format_x))
    ax.yaxis.set_major_formatter(plt.FuncFormatter(format_y))
    ax.set_xlabel("Longitude",fontsize=22)
    ax.set_ylabel("Latitude",fontsize=22)

    #Set tick label size
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize(16)

## Loading in the GPS files

The code as written assumed all the GPS data is in .gpx files in the working directory.

I downloaded my running history from Garmin using [this](https://github.com/pe-st/garmin-connect-export). Requesting your whole running history from Garmin directly (see [here](https://support.garmin.com/en-CA/?faq=W1TvTPW8JZ6LfJSfK512Q8)) should work as well, but the extraction function will need to be tweaked as the files are provided as .fit files by default and the file names are less descriptive.

If you running data is natively hosted elsewhere (e.g. Strava, MapMyRun) then you're on your own, sorry!

In [ ]:
#Find the runs you'd like to plot.
runs = glob.glob('*Toronto*.gpx')

#Extract the data from the GPX files and align the runs in time.
#Can take a little while to run depending on how many runs you have.
tracks       = [extract_gpx_data(run) for run in runs]
aligned_gdfs = [time_aligned_track(track) for track in tracks]

# Calculate the duration of each run
gdfs_with_duration = [
    (gdf, gdf['time'].iloc[-1] - gdf['time'].iloc[0]) \
                for gdf in aligned_gdfs
]

# Sort by duration so that shorter runs are plotted under longers ones
gdfs_with_duration.sort(key=lambda x: x[1])

aligned_gdfs = [gdf for gdf, _ in gdfs_with_duration]

fade_duration = 1800  # seconds (how long the trail should be)

## Constructing the animation

A lot of code here, but it sets up the animation. Can take a while if many runs are being animated

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot invisible background traces
for gdf in aligned_gdfs:
    gdf.plot(ax=ax,alpha=0.00)

# Create transformer from WGS84 (lat/lon) to Web Mercator (meters)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

#Lon/Lat coordinate ranges for the frame (degrees)
lonlow, latlow = -79.55, 43.59
lonhi, lathi = -79.25, 43.75

#Set lon/lat limits
xlow, ylow = transformer.transform(lonlow, latlow)
xhigh, yhigh = transformer.transform(lonhi, lathi)
ax.set_xlim(xlow, xhigh)
ax.set_ylim(ylow, yhigh)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

#Convert the axis labels from meters to degrees
#Not needed if there are no axis labels
#meters_to_lonlat(ax)

# Remove tick labels
ax.set_xticks([])
ax.set_yticks([])

# Remove axis labels
ax.set_xlabel('')
ax.set_ylabel('')

solid_colors = ['red', 'blue', 'green', 'orange', 'purple', 'black']
points = []
line_collections = []
trail_collections = []

# Draw scale bar in meters (adjust as needed)
scale_length = 5000  # meters

ax.add_line(Line2D([0.7, 0.7 + scale_length/(ax.get_xlim()[1] - ax.get_xlim()[0]) ], [0.12,0.12], transform=ax.transAxes,
                   color='black', linewidth=3, zorder=10))

# Label it
ax.text(0.7 + 0.5*scale_length/(ax.get_xlim()[1] - ax.get_xlim()[0]), 0.15, f'{scale_length/1000} km', transform=ax.transAxes,
        ha='center', va='bottom', fontsize=16, zorder=10,
        bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

# Add north arrow
ax.annotate('N',
            xy=(0.05, 0.95), xytext=(0.05, 0.8), xycoords='axes fraction',
            arrowprops=dict(facecolor='black', width=2, headwidth=8),
            ha='center', fontsize=16, zorder=10,
            bbox=dict(facecolor='none', edgecolor='none', alpha=0.7))

for i, gdf in enumerate(aligned_gdfs):
    
    color = solid_colors[i % len(solid_colors)]
    
    line, = ax.plot([], [], color=color, linewidth=2)
    
    line_collections.append((line, gdf, None))
    
    pt, = ax.plot([], [], 'o', color=solid_colors[i % len(solid_colors)], \
                    markersize=6, markeredgecolor='black',markeredgewidth=1)
    
    points.append(pt)

    # Add fading trail (empty for now)
    trail_lc = LineCollection([], linewidth=2, zorder=0)
    ax.add_collection(trail_lc)
    trail_collections.append((trail_lc, gdf))

#Initialize elapsed time text
elapsed_time_text = ax.text(
    0.6, 0.03, '', transform=ax.transAxes,
    ha='left', va='bottom', fontsize=16,
    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2'),
    zorder=10
)

ax.set_aspect('equal')
#ax.autoscale()

# Animation functions
def init():
    for line, _, _ in line_collections:
        line.set_data([], [])
    for pt in points:
        pt.set_data([], [])
    for trail_lc, _ in trail_collections:
        trail_lc.set_segments([])

    return points + [obj[0] for obj in line_collections]

def update(frame_time):
    artists = []

    # Update elapsed time textbox
    elapsed_seconds = int(frame_time - frame_times[0])
    elapsed_str = str(datetime.timedelta(seconds=elapsed_seconds))
    elapsed_time_text.set_text(f"Time Elapsed: {elapsed_str}")

    for i, (obj, gdf, segments) in enumerate(line_collections):

        #Determine how far each run has progressed
        mask = gdf['time'] <= frame_time
        visible = gdf[mask]

        if visible.empty:
            continue

        x = visible.geometry.x
        y = visible.geometry.y
        obj.set_data(x, y)
        obj.set_alpha(0.2)

        # Update the moving point
        pt = points[i]
        pt.set_data([visible.geometry.x.iloc[-1]], \
                    [visible.geometry.y.iloc[-1]])

        # Update fading trail
        trail_lc, trail_gdf = trail_collections[i]
        trail_lc.set_alpha(None) 

        recent_mask = (trail_gdf['time'] <= frame_time) \
                       & (trail_gdf['time'] >= frame_time - fade_duration)

        recent = trail_gdf[recent_mask]

        if len(recent) >= 2:
            coords = np.array([(p.x, p.y) for p in recent.geometry])
            segments = np.array([coords[:-1], coords[1:]]).transpose(1, 0, 2)

            num_segments = len(segments)
            alphas = np.linspace(0.2, 1.0, num_segments)

            color = solid_colors[i % len(solid_colors)]
            base_rgb = mcolors.to_rgb(color)
            colors = [(base_rgb[0], base_rgb[1], base_rgb[2], a) for a in alphas]

            trail_lc.set_alpha(None)
            trail_lc.set_segments(segments)
            trail_lc.set_color(colors)
        else:
            trail_lc.set_segments([])

        artists.extend([pt, obj[0] if isinstance(obj, tuple) \
                        else obj, trail_lc])

    return artists

#Find the maximum duration across all runs
max_time = max([gdf['time'].max() for gdf in aligned_gdfs])

#How many real seconds of running per frame
real_seconds_per_frame = 100.0
#How many frames per second in the video
video_ms_per_frame = 50.
#How long to pause at the start and end of the video
frame_pause_ms = 1000.

#Set up the frames at which to plot the data
pause_frames = int(frame_pause_ms / video_ms_per_frame)
frame_times = np.arange(0, max_time, real_seconds_per_frame)

frame_times = (
    [frame_times[0]]*pause_frames +
    list(frame_times) +
    [frame_times[-1]]*pause_frames
)

ani = FuncAnimation(fig, update, frames=frame_times, init_func=init, blit=False, interval=video_ms_per_frame)

## Time to animate!

Can take a while to run depending on how many runs you're animating

In [ ]:
from IPython.display import HTML
HTML(ani.to_html5_video())